# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f71c001f3d0>
├── 'a' --> tensor([[-0.8520,  0.7400,  1.0168],
│                   [-0.7716, -0.1605, -0.0578]])
└── 'x' --> <FastTreeValue 0x7f71c001fdf0>
    └── 'c' --> tensor([[ 0.0490, -1.2201,  0.3432,  1.0398],
                        [-2.1274,  0.6964,  0.6915, -0.7288],
                        [-0.8029,  1.7824, -0.2082, -0.7766]])

In [4]:
t.a

tensor([[-0.8520,  0.7400,  1.0168],
        [-0.7716, -0.1605, -0.0578]])

In [5]:
%timeit t.a

59.8 ns ± 0.0503 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f71c001f3d0>
├── 'a' --> tensor([[-0.1601, -0.6404,  2.6733],
│                   [ 1.1270, -0.2742, -1.1442]])
└── 'x' --> <FastTreeValue 0x7f71c001fdf0>
    └── 'c' --> tensor([[ 0.0490, -1.2201,  0.3432,  1.0398],
                        [-2.1274,  0.6964,  0.6915, -0.7288],
                        [-0.8029,  1.7824, -0.2082, -0.7766]])

In [7]:
%timeit t.a = new_value

59.4 ns ± 0.0343 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.8520,  0.7400,  1.0168],
               [-0.7716, -0.1605, -0.0578]]),
    x: Batch(
           c: tensor([[ 0.0490, -1.2201,  0.3432,  1.0398],
                      [-2.1274,  0.6964,  0.6915, -0.7288],
                      [-0.8029,  1.7824, -0.2082, -0.7766]]),
       ),
)

In [10]:
b.a

tensor([[-0.8520,  0.7400,  1.0168],
        [-0.7716, -0.1605, -0.0578]])

In [11]:
%timeit b.a

54.3 ns ± 0.0533 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.3892, -0.5003,  0.4958],
               [-0.4589, -1.2427,  0.7697]]),
    x: Batch(
           c: tensor([[ 0.0490, -1.2201,  0.3432,  1.0398],
                      [-2.1274,  0.6964,  0.6915, -0.7288],
                      [-0.8029,  1.7824, -0.2082, -0.7766]]),
       ),
)

In [13]:
%timeit b.a = new_value

480 ns ± 0.0897 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

895 ns ± 0.169 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.6 µs ± 31.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

137 µs ± 730 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

138 µs ± 544 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f71c0012100>
├── 'a' --> tensor([[[-0.8520,  0.7400,  1.0168],
│                    [-0.7716, -0.1605, -0.0578]],
│           
│                   [[-0.8520,  0.7400,  1.0168],
│                    [-0.7716, -0.1605, -0.0578]],
│           
│                   [[-0.8520,  0.7400,  1.0168],
│                    [-0.7716, -0.1605, -0.0578]],
│           
│                   [[-0.8520,  0.7400,  1.0168],
│                    [-0.7716, -0.1605, -0.0578]],
│           
│                   [[-0.8520,  0.7400,  1.0168],
│                    [-0.7716, -0.1605, -0.0578]],
│           
│                   [[-0.8520,  0.7400,  1.0168],
│                    [-0.7716, -0.1605, -0.0578]],
│           
│                   [[-0.8520,  0.7400,  1.0168],
│                    [-0.7716, -0.1605, -0.0578]],
│           
│                   [[-0.8520,  0.7400,  1.0168],
│                    [-0.7716, -0.1605, -0.0578]]])
└── 'x' --> <FastTreeValue 0x7f71c0012370>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.5 µs ± 31.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f71c008b9d0>
├── 'a' --> tensor([[-0.8520,  0.7400,  1.0168],
│                   [-0.7716, -0.1605, -0.0578],
│                   [-0.8520,  0.7400,  1.0168],
│                   [-0.7716, -0.1605, -0.0578],
│                   [-0.8520,  0.7400,  1.0168],
│                   [-0.7716, -0.1605, -0.0578],
│                   [-0.8520,  0.7400,  1.0168],
│                   [-0.7716, -0.1605, -0.0578],
│                   [-0.8520,  0.7400,  1.0168],
│                   [-0.7716, -0.1605, -0.0578],
│                   [-0.8520,  0.7400,  1.0168],
│                   [-0.7716, -0.1605, -0.0578],
│                   [-0.8520,  0.7400,  1.0168],
│                   [-0.7716, -0.1605, -0.0578],
│                   [-0.8520,  0.7400,  1.0168],
│                   [-0.7716, -0.1605, -0.0578]])
└── 'x' --> <FastTreeValue 0x7f71c0016790>
    └── 'c' --> tensor([[ 0.0490, -1.2201,  0.3432,  1.0398],
                        [-2.1274,  0.6964,  0.6915, -0.7288],
                 

In [23]:
%timeit t_cat(trees)

30.3 µs ± 45.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58.4 µs ± 77.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.8520,  0.7400,  1.0168],
                [-0.7716, -0.1605, -0.0578]],
       
               [[-0.8520,  0.7400,  1.0168],
                [-0.7716, -0.1605, -0.0578]],
       
               [[-0.8520,  0.7400,  1.0168],
                [-0.7716, -0.1605, -0.0578]],
       
               [[-0.8520,  0.7400,  1.0168],
                [-0.7716, -0.1605, -0.0578]],
       
               [[-0.8520,  0.7400,  1.0168],
                [-0.7716, -0.1605, -0.0578]],
       
               [[-0.8520,  0.7400,  1.0168],
                [-0.7716, -0.1605, -0.0578]],
       
               [[-0.8520,  0.7400,  1.0168],
                [-0.7716, -0.1605, -0.0578]],
       
               [[-0.8520,  0.7400,  1.0168],
                [-0.7716, -0.1605, -0.0578]]]),
    x: Batch(
           c: tensor([[[ 0.0490, -1.2201,  0.3432,  1.0398],
                       [-2.1274,  0.6964,  0.6915, -0.7288],
                       [-0.8029,  1.7824, -0.2082, -0.7766]],
         

In [26]:
%timeit Batch.stack(batches)

77.6 µs ± 143 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.8520,  0.7400,  1.0168],
               [-0.7716, -0.1605, -0.0578],
               [-0.8520,  0.7400,  1.0168],
               [-0.7716, -0.1605, -0.0578],
               [-0.8520,  0.7400,  1.0168],
               [-0.7716, -0.1605, -0.0578],
               [-0.8520,  0.7400,  1.0168],
               [-0.7716, -0.1605, -0.0578],
               [-0.8520,  0.7400,  1.0168],
               [-0.7716, -0.1605, -0.0578],
               [-0.8520,  0.7400,  1.0168],
               [-0.7716, -0.1605, -0.0578],
               [-0.8520,  0.7400,  1.0168],
               [-0.7716, -0.1605, -0.0578],
               [-0.8520,  0.7400,  1.0168],
               [-0.7716, -0.1605, -0.0578]]),
    x: Batch(
           c: tensor([[ 0.0490, -1.2201,  0.3432,  1.0398],
                      [-2.1274,  0.6964,  0.6915, -0.7288],
                      [-0.8029,  1.7824, -0.2082, -0.7766],
                      [ 0.0490, -1.2201,  0.3432,  1.0398],
                      [-2.1274,  

In [28]:
%timeit Batch.cat(batches)

142 µs ± 247 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

321 µs ± 2.91 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
